# PETs/TETs – Hyperledger Aries – Manufacturer 1 (Holder) 💼

In [1]:
%%javascript
document.title ='💼 Manufacturer1 Agent'

<IPython.core.display.Javascript object>

## PART 3: Connect with City to Analyze Data

**What:** -

**Why:** -

**How:** <br>


**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_manufacturer1.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
import libs.helpers as helpers
from libs.agent_connection_manager import CredentialHolder
import os
from termcolor import colored

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer1 Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer1_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer1_agent.get_credentials()

Successfully initiated AgentConnectionManager for a(n) Holder ACA-PY agent


{'results': [{'referent': 'M1-isManufacturer',
   'attrs': {'manufacturerName': 'undisclosedManufacturer1',
    'manufacturerCountry': 'Germany',
    'manufacturerCity': 'Berlin',
    'isManufacturer': 'TRUE'},
   'schema_id': 'au8Y1rjsZyz9jbpr3Lk54:2:certify-manufacturer:0.0.1',
   'cred_def_id': 'au8Y1rjsZyz9jbpr3Lk54:3:CL:250893:default',
   'rev_reg_id': None,
   'cred_rev_id': None}]}

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the manufacturer1 requests a connection with the Authority to be certified as an official city agency. Thus, the manufacturer1 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [9]:
# Variables
alias = "undisclosedM1"
auto_accept = True
label = None

# Receive connection invitation
connection_id = manufacturer1_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept, label=label)

Please enter invitation received by external agent.


Invitation:  {     '@id': '71dff343-d2ca-46e4-82bf-c04774941664',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'City',     'recipientKeys': ['CBUT7cYooo3F39f7P3LgdqZABppWcjHKCdP25z5wJNmN'],     'serviceEndpoint': 'https://898696e3e9e9.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb4d720f-5648-4181-b1dc-a4535218743a
State :  invitation (invitation-received)
Routing State : none
Connection with :  City
Their Role :  inviter
---------------------------------------------------------------------


Accept invitation cb4d720f-5648-4181-b1dc-a4535218743a? [yes/no] yes



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb4d720f-5648-4181-b1dc-a4535218743a
State :  request (request-sent)
Routing State : none
Connection with :  City
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb4d720f-5648-4181-b1dc-a4535218743a
State :  response (response-received)
Routing State : none
Connection with :  City
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  cb4d720f-5648-4181-b1dc-a4535218743a
State :  active (completed)
Routing State : none
Connection with :  City
Their Role :  inviter
---------------------------------------------------------------------

Connection ID

<div style="font-size: 25px"><center><b>Break Point 2/3/4</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.3</b></center>

---

### 3 – Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [ ]:
# Step 2

# Optional Query parameters
verifier_connection_id = connection_id
thread_id = None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)
pprint(proof_records_response)

In [ ]:
# Fetch records from response that were sent via connection_id
presentation_records = [r for r in proof_records_response["results"] if (r["connection_id"] == connection_id) is True] 

# Process only the first record:
presentation_record = presentation_records[0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [ ]:
def get_proof_request_requirements(presentation_record):
    """
    Returns dictionary with {<required-attribute>: <restrictions-of-attribute>} from presentation record
    """
    # Setup
    restrictions = {}
    presentation_request = presentation_record["presentation_request"]
    
    # Get required attributes and requirements for the individual attributes
    for attr_key, attr_val in presentation_request["requested_attributes"].items():
        restrictions[attr_val["name"]] = {}
        restrictions[attr_val["name"]]["requirements"] = attr_val["restrictions"][0] 
        restrictions[attr_val["name"]]["request_attr_name"] = attr_key
        
    return restrictions


def get_suitable_credentials(credentials_all, requirements):
    """
    Finds credentials in credentials_all that satisfy the requirements provided by the relying party.
    Returns dictionary with: {<attribute-name>: <suitable-credential>}, where the suitable-credential satisfies all requirements 
    """
    # Setup
    relevant_credentials = {}
    revealed = {}
    credentials = credentials_all["results"]
    
    # Iterate through attribute name and attribute requirements of relying party
    for name, conditions in requirements.items():
        
        req = conditions["requirements"]
        req_name = conditions["request_attr_name"]
        
        # Break if the required attribute name is not in any credential, or if all requirements (e.g., schema_id) are not within one credential
        if (any(name in cred["attrs"] for cred in credentials) is False) or (any(r in cred.keys() for r in req for cred in credentials) is False):
            continue

        # Iterate through credentials
        for cred in credentials:

            # Verify if requirement value (r_val) and credential value (cred[r_key]) match for required attribute (r_key)
            for r_key, r_val in req.items():
                try:
                    # Append cred to relevant_credentials if all requirements match
                    if (cred[r_key] == r_val) is True:
                        relevant_credentials[name] = cred
                        print(f"Attribute request for '{name}' can be satisfied by Credential with Referent -- {cred['referent']}")
                        revealed[req_name] = {"cred_id": cred["referent"], "revealed": True}
                except:
                    pass

    return relevant_credentials, revealed

In [ ]:
# Get requirements of proof request
proof_attribute_requirements = get_proof_request_requirements(presentation_record)

# Verify all credentials of M1 and verify if they satisfy the requirements defined in proof_attribute_requirements
credentials = await agent_controller.credentials.get_all()
suitable_credentials, revealed = get_suitable_credentials(credentials, proof_attribute_requirements)
#pprint(credentials)
pprint(revealed)

In [ ]:
# select credentials to provide for the proof
#credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
#print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")

#attribute_by_reft = {}
#revealed = {}
#self_attested = {}
#predicates = {}

In [ ]:
predicates = {}
self_attested = {}


In [ ]:
# Note we are working on a friendlier api to abstract this away

print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(presentation)

## Send Presentation

A presentation is sent in represent to a presentation record that has previously been created.

In [ ]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
# Imports
import torch as th
import syft as sy

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

In [ ]:
# Copy and paste client_id and ping the city!

In [ ]:
# Step 3: Create duet server
duet_m1 = sy.duet()

## 2 - Secure Multi-Party Computation

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
# Step 8: test there is no data in the server atm
duet_m1.store.pandas

In [ ]:
age_data = th.tensor([50,23,72,83])
age_data = age_data.tag("test_data_age")
age_data = age_data.describe("description for age_data")
age_data_pointer = age_data.send(duet_m1, pointable=True)

In [ ]:
duet_m1.store

In [ ]:
duet_m1.store.pandas

In [ ]:
duet_m1.id

In [ ]:
duet_m1.name

In [ ]:
duet_m1.target_id

In [ ]:
duet_m1.network_id

In [ ]:
#duet_city.store.pandas

In [ ]:
# Step 9: Publish secret data

x = torch.tensor([50,60,77]) # define torch
x.send(duet_m1, pointable=True, tags=["TEST_#1_from_m1_to_ds"], description="Dummy data") # send torch to duet_m1

x = torch.tensor([[1,2], [3,4]])
x.send(duet_m1, pointable=True, tags=["TEST_#2_from_m1_to_ds"], description="Test to multiply matrices")

In [ ]:
# Step 10: verify that data was uploaded to the store
duet_m1.store.pandas

In [ ]:
# Step 11: Authorize the requests ds can make
duet_m1.requests.add_handler(action="accept")

In [ ]:
# Step 11: Authorize the requests ds can make
duet_city.requests.add_handler(action="accept")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()